REQUIRED LIBRARY

In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import cross_val_score, KFold

LOADING DATASET

In [2]:
def extract_face(filename, required_size=(256, 256)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
# resize pixels to the model size
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [3]:
import os

def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = os.path.join(dir, filename)
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(dir):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = os.path.join(dir, subdir)
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces), subdir))  # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)


In [4]:
# load train dataset
trainX, trainy = load_dataset(r"C:\Users\WELCOME\Desktop\jupiter test\Destination\train")
print(trainX.shape, trainy.shape)

loaded 16 sample for class: 1
loaded 16 sample for class: 10
loaded 16 sample for class: 11
loaded 16 sample for class: 12
loaded 16 sample for class: 13
loaded 16 sample for class: 14
loaded 16 sample for class: 15
loaded 16 sample for class: 16
loaded 16 sample for class: 18
loaded 16 sample for class: 19
loaded 16 sample for class: 2
loaded 16 sample for class: 20
loaded 15 sample for class: 21
loaded 16 sample for class: 3
loaded 16 sample for class: 4
loaded 16 sample for class: 5
loaded 16 sample for class: 6
loaded 16 sample for class: 7
loaded 16 sample for class: 8
loaded 16 sample for class: 9
loaded 16 sample for class: B15DCKT150
loaded 16 sample for class: B17DCCN318
loaded 16 sample for class: B17DCCN456
loaded 16 sample for class: B17DCCN610
loaded 16 sample for class: B18DCAT009
loaded 16 sample for class: B18DCAT010
loaded 16 sample for class: B18DCAT018
loaded 16 sample for class: B18DCAT026
loaded 16 sample for class: B18DCAT030
loaded 16 sample for class: B18DCAT032

In [5]:
# load test dataset
testX, testy = load_dataset(r"C:\Users\WELCOME\Desktop\jupiter test\Destination\test")
print(testX.shape, testy.shape)

loaded 4 sample for class: 1
loaded 4 sample for class: 10
loaded 4 sample for class: 11
loaded 4 sample for class: 12
loaded 4 sample for class: 13
loaded 4 sample for class: 14
loaded 4 sample for class: 15
loaded 4 sample for class: 16
loaded 4 sample for class: 18
loaded 4 sample for class: 19
loaded 4 sample for class: 2
loaded 4 sample for class: 20
loaded 4 sample for class: 21
loaded 4 sample for class: 3
loaded 4 sample for class: 4
loaded 4 sample for class: 5
loaded 4 sample for class: 6
loaded 4 sample for class: 7
loaded 4 sample for class: 8
loaded 4 sample for class: 9
loaded 4 sample for class: B15DCKT150
loaded 4 sample for class: B17DCCN318
loaded 4 sample for class: B17DCCN456
loaded 4 sample for class: B17DCCN610
loaded 4 sample for class: B18DCAT009
loaded 4 sample for class: B18DCAT010
loaded 4 sample for class: B18DCAT018
loaded 4 sample for class: B18DCAT026
loaded 4 sample for class: B18DCAT030
loaded 4 sample for class: B18DCAT032
loaded 4 sample for class: B1

COMPRESS AND SAVE TRAIN AND TEST DATA

In [6]:
np.savez_compressed('new_face.npz', trainX, trainy, testX, testy)

LOAD SAVED COMPRESSED MODEL

In [7]:
data = np.load("new_face.npz")
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (4978, 256, 256, 3) (4978,) (1248, 256, 256, 3) (1248,)


In [8]:
trainx, valid = train_test_split(trainX, test_size=0.1, random_state=42, shuffle=True)

In [9]:
print("number of image in train dataset : %s" %(len(trainx)))

print("number of image in Valid  dataset : %s" %(len(valid)))

number of image in train dataset : 4480
number of image in Valid  dataset : 498


In [10]:
y_train, y_valid = train_test_split(trainy, test_size=0.1, random_state=42, shuffle=True)

In [11]:
print("number of image in train dataset : %s" %(len(y_train)))

print("number of image in Valid dataset : %s" %(len(y_valid)))

number of image in train dataset : 4480
number of image in Valid dataset : 498


In [12]:
np.savez_compressed('extracted_face.npz', trainx, y_train, valid, y_valid,testX, testy)

In [13]:
data = np.load("extracted_face.npz")
trainx, y_train, valid, y_valid,testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

Loaded:  (4480, 256, 256, 3) (4480,) (498, 256, 256, 3) (498,) (1248, 256, 256, 3) (1248,)


FACE EMBEDDING WITH  FACENET

In [14]:
from keras_facenet import FaceNet
facenet_model=FaceNet();
print('Loaded model')

Loaded model


In [15]:
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

Loaded:  (4480, 256, 256, 3) (4480,) (498, 256, 256, 3) (498,) (1248, 256, 256, 3) (1248,)


In [16]:
def get_embedding(model, face):
    
    face = face.astype('float32')

    #mean, std = face.mean(), face.std()
    #face = (face-mean)/std
    
    sample = np.expand_dims(face, axis=0)
    
    yhat = model.embeddings(sample)
    return yhat[0]

CONVERT EACH FACE IN THE TRAINING SET

In [17]:
emdTrainX = list()
for face in trainx:
    emd = get_embedding(facenet_model, face)
    emdTrainX.append(emd)
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)          
embValid = list()
for face in valid:
    emd = get_embedding(facenet_model,face)
    embValid.append(emd)
embValid = np.asarray(embValid)
print(embValid.shape)

1/1 [==============================] - 0s 312ms/step
(4480, 512)
1/1 [==============================] - 0s 346ms/step
(498, 512)


CONVERT EACH FACE IN THE TESTING SET

In [18]:
emdTestX = list()
for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

1/1 [==============================] - 1s 543ms/step
(1248, 512)


COMPRESS AND SAVE TRAIN AND TESTING MODEL

In [19]:
np.savez_compressed('face_embeddings.npz', emdTrainX, y_train, embValid, y_valid, emdTestX, testy)

LOAD SAVED COMPRESSED FILE

In [20]:
data = np.load('face_embeddings.npz')
emdTrainX, y_train, embValid, y_valid, emdTestX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']

In [21]:
print('Loaded: ', emdTrainX.shape, y_train.shape, embValid.shape, y_valid.shape, emdTestX.shape, testy.shape)

Loaded:  (4480, 512) (4480,) (498, 512) (498,) (1248, 512) (1248,)


LABLE ENCODDING

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle
import numpy as np

# Assuming you have defined emdTrainX, embValid, emdTestX, y_train, y_validation, and y_test

print("Dataset: train=%d, validation=%d, test=%d" % (emdTrainX.shape[0], embValid.shape[0], emdTestX.shape[0]))

# Normalize input vectors
in_encoder = Normalizer(norm='l2')
emdTrainX_norm = in_encoder.transform(emdTrainX)
embValid_norm = in_encoder.transform(embValid)
emdTestX_norm = in_encoder.transform(emdTestX)

# Label encode targets
out_encoder = LabelEncoder()

# Concatenate y_train, y_validation, and y_test
encoder_arr = np.append(np.append(y_train, y_valid), testy)

out_encoder.fit(encoder_arr)



Dataset: train=4480, validation=498, test=1248


LabelEncoder()

ENCODING

In [23]:
trainy_enc = out_encoder.transform(y_train)
y_valid_enc = out_encoder.transform(y_valid)
testy_enc = out_encoder.transform(testy)

FACE RECOGNITION WITH SVC

In [24]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np
import pickle

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
model = SVC(kernel='poly', probability=True)  # You can choose a different kernel if needed

train_accuracies = []
test_accuracies = []

train_recalls = []
train_precisions = []
train_f1_scores = []

test_recalls = []
test_precisions = []
test_f1_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(emdTrainX)):
    train_X_fold, test_X_fold = emdTrainX[train_index], emdTrainX[test_index]
    train_y_fold, test_y_fold = trainy_enc[train_index], trainy_enc[test_index]
    
    model.fit(train_X_fold, train_y_fold)
    
    train_accuracy = model.score(train_X_fold, train_y_fold) * 100
    test_accuracy = model.score(test_X_fold, test_y_fold) * 100
    
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    
    y_pred_train = model.predict(train_X_fold)
    y_pred_test = model.predict(test_X_fold)
    
    recall_train = recall_score(train_y_fold, y_pred_train, average='weighted')
    precision_train = precision_score(train_y_fold, y_pred_train, average='weighted')
    f1_train = f1_score(train_y_fold, y_pred_train, average='weighted')
    
    recall_test = recall_score(test_y_fold, y_pred_test, average='weighted')
    precision_test = precision_score(test_y_fold, y_pred_test, average='weighted')
    f1_test = f1_score(test_y_fold, y_pred_test, average='weighted')
    
    train_recalls.append(recall_train)
    train_precisions.append(precision_train)
    train_f1_scores.append(f1_train)
    
    test_recalls.append(recall_test)
    test_precisions.append(precision_test)
    test_f1_scores.append(f1_test)
    
    print(f"Fold {fold + 1} - Training Accuracy: {train_accuracy:.2f}% | Testing Accuracy: {test_accuracy:.2f}%")
    print(f"Fold {fold + 1} - Training Recall: {recall_train:.3f} | Training Precision: {precision_train:.3f} | Training F1-score: {f1_train:.3f}")
    print(f"Fold {fold + 1} - Testing Recall: {recall_test:.3f} | Testing Precision: {precision_test:.3f} | Testing F1-score: {f1_test:.3f}")

# Calculate the average and standard deviation of metrics
average_train_recall = np.mean(train_recalls)
std_train_recall = np.std(train_recalls)
average_test_recall = np.mean(test_recalls)
std_test_recall = np.std(test_recalls)

average_train_precision = np.mean(train_precisions)
std_train_precision = np.std(train_precisions)
average_test_precision = np.mean(test_precisions)
std_test_precision = np.std(test_precisions)

average_train_f1 = np.mean(train_f1_scores)
std_train_f1 = np.std(train_f1_scores)
average_test_f1 = np.mean(test_f1_scores)
std_test_f1 = np.std(test_f1_scores)

print("\nTraining Metrics:")
print(f'Average Training Recall: {average_train_recall:.3f} +/- {std_train_recall:.3f}')
print(f'Average Training Precision: {average_train_precision:.3f} +/- {std_train_precision:.3f}')
print(f'Average Training F1-score: {average_train_f1:.3f} +/- {std_train_f1:.3f}')

print("\nTesting Metrics:")
print(f'Average Testing Recall: {average_test_recall:.3f} +/- {std_test_recall:.3f}')
print(f'Average Testing Precision: {average_test_precision:.3f} +/- {std_test_precision:.3f}')
print(f'Average Testing F1-score: {average_test_f1:.3f} +/- {std_test_f1:.3f}')

# Train the model on the full training data
model.fit(emdTrainX, trainy_enc)

# Save the trained model
filename = 'Cross_benchmark.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)


Fold 1 - Training Accuracy: 99.67% | Testing Accuracy: 98.21%
Fold 2 - Training Accuracy: 99.83% | Testing Accuracy: 97.54%
Fold 3 - Training Accuracy: 99.89% | Testing Accuracy: 98.21%
Fold 4 - Training Accuracy: 99.58% | Testing Accuracy: 96.54%
Fold 5 - Training Accuracy: 99.78% | Testing Accuracy: 97.43%

Training Accuracies:
Fold 1: 99.67%
Fold 2: 99.83%
Fold 3: 99.89%
Fold 4: 99.58%
Fold 5: 99.78%

Testing Accuracies:
Fold 1: 98.21%
Fold 2: 97.54%
Fold 3: 98.21%
Fold 4: 96.54%
Fold 5: 97.43%

Average Training Accuracy: 99.75%
Standard Deviation of Training Accuracy: 0.11%
Average Testing Accuracy: 97.59%
Standard Deviation of Testing Accuracy: 0.62%


In [25]:
#loaded_model = pickle.load(open('Cross_benchmark.sav', 'rb'))